# QA Retreival 

https://colab.research.google.com/drive/1zG1R08TBikG05ecF8et4vi_1F9xutY-6?usp=sharing#scrollTo=jVWgPJXs1yRq

Local Model used is google/flan-t5-xl 

- https://huggingface.co/google/flan-t5-xl


Local Embeddings Used

-


In [ ]:
# !pip -q install langchain tiktoken chromadb pypdf transformers InstructorEmbedding
# !pip -q install accelerate bitsandbytes 

In [ ]:
# !wget -q https://www.dropbox.com/s/zoj9rnm7oyeaivb/new_papers.zip
# !unzip -q new_papers.zip -d new_papers

In [ ]:
# WORKS

# import torch
# import transformers
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# # torch_dtype=torch.float16, low_cpu_mem_usage=True,
                                            
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large",
#                                               load_in_8bit=False,
#                                               device_map='cuda:0'
#                                               )


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("Writer/palmyra-base")
# model = AutoModelForCausalLM.from_pretrained("Writer/palmyra-base")

In [ ]:
# NOT WORKING

# import torch
# import transformers
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl",
#                                               load_in_4bit=True,
#                                               device_map='auto',
#                                             #   torch_dtype=torch.float16,
#                                             #   low_cpu_mem_usage=True,
                                            
#                                               )

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

api_key = open('./hf_key.txt').read()

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=api_key)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=api_key)

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=2048,
    temperature=0,
    repetition_penalty=1.15,
    batch_size=16
)

# pipe = pipeline(
#     # "text2text-generation",
#     "text-generation",
#     model=model, 
#     #tokenizer=tokenizer, 
#     max_length=2048,
#     temperature=0,
#     # top_p=0.95,
#     repetition_penalty=1.15,
#     # batch_size=16
# )

# pipe = pipeline("text-generation", model="TheBloke/falcon-40b-instruct-GPTQ", trust_remote_code=True,
#                max_length=2048,
#                 temperature=0,
#                 # top_p=0.95,
#                 repetition_penalty=1.15)


local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# from transformers import pipeline

# pipe2 = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")

# local_llm2 = HuggingFacePipeline(pipeline=pipe2)

Check is LLM works

In [ ]:
print(local_llm.invoke('What is the capital of the canada?'))

# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files - PDFs
- ChromaDB
- Local LLM
- Instuctor Embeddings

## Setting up LangChain 


In [ ]:
import os

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

## Load multiple and process documents

In [ ]:
# Load and process the text files
# loader = TextLoader('single_text_file.pdf')
loader = DirectoryLoader('./new_papers/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [ ]:
len(documents)

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

## HF Instructor Embeddings

In [ ]:

from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cuda:1"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
                                                      # model_kwargs={"device": "cuda:1"})

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'news_papers/db'

## Here is the nmew embeddings being used
embedding = hf

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

vectordb.persist()

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=local_llm, 
                                  chain_type="refine", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "What is Flash attention?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

In [ ]:
query = "What does IO-aware mean?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)
# llm_response

In [ ]:
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
query = "What are the differences between REALM and RAG?"
llm_response = qa_chain(query)
process_llm_response(llm_response)